In [2]:
import pandas as pd
import numpy as np
import glob
import os


In [177]:
# creating data frames

import pandas as pd
import numpy as np
import glob
import os

path = r'/Users/tom/Documents/Coding/AiCore/Projects/4. Football Match Outcome Predictor /Results'
all_files = glob.glob(os.path.join(path, "*"))

df_list = []

for path in all_files:
    all_csv_files = glob.glob(os.path.join(path, "*.csv"))
    df_name = (path.split("/")[9])
    df_list.append(f"{df_name}_df")
    globals()[f"{df_name}_df"] = pd.concat((pd.read_csv(f) for f in all_csv_files), ignore_index=True)

result_df = pd.concat((globals()[df] for df in df_list), ignore_index=True)

info_path = r'/Users/tom/Documents/Coding/AiCore/Projects/4. Football Match Outcome Predictor /Other/Match_Info.csv'
match_info_df = pd.read_csv(info_path)

team_path = r'/Users/tom/Documents/Coding/AiCore/Projects/4. Football Match Outcome Predictor /Other/Team_Info.csv'
team_info_df = pd.read_csv(team_path)


In [109]:
result_df

,Home_Team,Away_Team,Result,Link,Season,Round,League,Unnamed: 0
0,Watford,Middlesbrough,1-0,https://www.besoccer.com/match/watford-fc/midd...,2021,1,championship,NaN
1,Birmingham City,Brentford,1-0,https://www.besoccer.com/match/birmingham-city...,2021,1,championship,NaN
2,Wycombe Wanderers,Rotherham United,0-1,https://www.besoccer.com/match/wycombe-wandere...,2021,1,championship,NaN
3,AFC Bournemouth,Blackburn Rovers,3-2,https://www.besoccer.com/match/afc-bournemouth...,2021,1,championship,NaN
4,Barnsley,Luton Town,0-1,https://www.besoccer.com/match/barnsley-fc/lut...,2021,1,championship,NaN
...,...,...,...,...,...,...,...,...
146636,Pescara,Padova,1-2,https://www.besoccer.com/match/pescara-calcio/...,1997,38,serie_b,NaN
146637,Genoa,Palermo FC,4-1,https://www.besoccer.com/match/genoa/palermo/1...,1997,38,serie_b,NaN
146638,Torino,Ravenna FC,0-4,https://www.besoccer.com/match/torino-fc/raven...,1997,38,serie_b,NaN
146639,Salernitana,Reggina,1-3,https://www.besoccer.com/match/salernitana-cal...,1997,38,serie_b,NaN


In [178]:
# cleaning the match outcome data

result_df["Home_Team"] = result_df["Home_Team"].astype('category')
result_df["Away_Team"] = result_df["Away_Team"].astype('category')
result_df["Result"] = result_df["Result"].astype('string')
result_df["Season"] = result_df["Season"].astype('int')
result_df["Round"] = result_df["Round"].astype('int')
result_df["League"] = result_df["League"].astype('string')
result_df["Link"] = result_df["Link"].astype('string')

result_df.drop("Unnamed: 0", axis = 1, inplace=True)

invalid_score_1 = (result_df.loc[: , "Result"].str.contains(":"))
invalid_score_2 = (result_df.loc[: , "Result"].str.len() > 5)
invalid_score_3 = (result_df.loc[: , "Result"].str.len() < 2)


result_df = result_df.replace("1 (0-0) 1", np.NaN)
result_df = result_df.replace("0 (0-0) 0", "0-0")
result_df = result_df.replace("3 (3-2) 2", "3-2")
result_df = result_df.replace("0 (0-1) 1", "0-1")

result_df[invalid_score_1] = np.NaN
result_df[invalid_score_2] = np.NaN
result_df[invalid_score_3] = np.NaN



result_df["Result"].unique()

result_df = result_df.dropna(subset=["Result"])

# remove end of the match links

result_df["Link"] = result_df["Link"].apply(lambda x: "".join(x.replace(x.split("/")[6], "")) + x.split("/")[6][0:4])


In [179]:
# cleaning match and team info data

match_info_df["Home_Yellow"] = match_info_df["Home_Yellow"].replace(np.NaN, 0)
match_info_df["Home_Red"] = match_info_df["Home_Red"].replace(np.NaN, 0)
match_info_df["Away_Yellow"] = match_info_df["Away_Yellow"].replace(np.NaN, 0)
match_info_df["Away_Red"] = match_info_df["Away_Red"].replace(np.NaN, 0)


match_info_df["Date_New"] = match_info_df["Date_New"].astype('datetime64')
match_info_df["Referee"] = match_info_df["Referee"].astype('string')
match_info_df["Home_Yellow"] = match_info_df["Home_Yellow"].astype('int')
match_info_df["Home_Red"] = match_info_df["Home_Red"].astype('int')
match_info_df["Away_Yellow"] = match_info_df["Away_Yellow"].astype('int')
match_info_df["Away_Red"] = match_info_df["Away_Red"].astype('int')
match_info_df["Link"] = match_info_df["Link"].astype('string')



# clean all match and team info data sets

match_info_df["Referee"] = match_info_df["Referee"].apply(lambda x: (x.replace("\r\n", "")).replace("Referee: ", ""))

match_info_df["Link"] = match_info_df["Link"].apply(lambda x: "https://www.besoccer.com" + x)

team_info_df["Capacity"] = team_info_df["Capacity"].apply(lambda x: x.replace(",", ""))

team_info_df["Capacity"] = team_info_df["Capacity"].astype('int')

# rename column in team info

team_info_df = team_info_df.rename(columns = {'Team' : 'Home_Team'})






In [180]:
team_info_df.info()




<class 'pandas.core.frame.DataFrame'>
RangeIndex: 544 entries, 0 to 543
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Home_Team  544 non-null    object
 1   City       544 non-null    object
 2   Country    544 non-null    object
 3   Stadium    447 non-null    object
 4   Capacity   544 non-null    int64 
 5   Pitch      447 non-null    object
dtypes: int64(1), object(5)
memory usage: 25.6+ KB


In [183]:
match_info_df["Link"].unique()

array(['https://www.besoccer.com/match/saarbrucken/stuttgarter-kickers/1990',
       'https://www.besoccer.com/match/sc-freiburg/unterhaching/1990',
       'https://www.besoccer.com/match/vfl-osnabruck/meppen/1990', ...,
       'https://www.besoccer.com/match/tranmere-rovers/wolverhampton/1999',
       'https://www.besoccer.com/match/sochaux/guingamp/2021',
       'https://www.besoccer.com/match/huddersfield-town-fc/crystal-palace-fc/1999'],
      dtype=object)

In [184]:
# adding features to outcome data


result_df["Home_Goals"] = result_df["Result"].apply(lambda x: x.split("-")[0])
result_df["Away_Goals"] = result_df["Result"].apply(lambda x: x.split("-")[1])
result_df["Home_Win"] = result_df["Home_Goals"] > result_df["Away_Goals"]
result_df["Away_Win"] = result_df["Home_Goals"] < result_df["Away_Goals"]




In [185]:
result_df

,Home_Team,Away_Team,Result,Link,Season,Round,League,Home_Goals,Away_Goals,Home_Win,Away_Win
0,Watford,Middlesbrough,1-0,https://www.besoccer.com/match/watford-fc/midd...,2021.0,1.0,championship,1,0,True,False
1,Birmingham City,Brentford,1-0,https://www.besoccer.com/match/birmingham-city...,2021.0,1.0,championship,1,0,True,False
2,Wycombe Wanderers,Rotherham United,0-1,https://www.besoccer.com/match/wycombe-wandere...,2021.0,1.0,championship,0,1,False,True
3,AFC Bournemouth,Blackburn Rovers,3-2,https://www.besoccer.com/match/afc-bournemouth...,2021.0,1.0,championship,3,2,True,False
4,Barnsley,Luton Town,0-1,https://www.besoccer.com/match/barnsley-fc/lut...,2021.0,1.0,championship,0,1,False,True
...,...,...,...,...,...,...,...,...,...,...,...
146636,Pescara,Padova,1-2,https://www.besoccer.com/match/pescara-calcio/...,1997.0,38.0,serie_b,1,2,False,True
146637,Genoa,Palermo FC,4-1,https://www.besoccer.com/match/genoa/palermo/1997,1997.0,38.0,serie_b,4,1,True,False
146638,Torino,Ravenna FC,0-4,https://www.besoccer.com/match/torino-fc/raven...,1997.0,38.0,serie_b,0,4,False,True
146639,Salernitana,Reggina,1-3,https://www.besoccer.com/match/salernitana-cal...,1997.0,38.0,serie_b,1,3,False,True


In [186]:
team_info_df

,Home_Team,City,Country,Stadium,Capacity,Pitch
0,Wattenscheid 09,Bochum-Wattenscheid,Germany,Lohrheidestadion,16233,Natural
1,Hertha BSC,Berlín,Germany,Olympiastadion Berlin,76065,Natural
2,Unterhaching,Unterhaching,Germany,Sportpark Unterhaching,15053,Natural
3,Fortuna Köln,Cologne,Germany,Südstadion,14944,Natural
4,MSV Duisburg,Duisburgo,Germany,Schauinsland-Reisen-Arena,31514,Natural
...,...,...,...,...,...,...
539,Latina,Latina,Italy,Stadio Comunale Domenico Francioni,8000,Natural
540,Virtus Entella,Chiavari,Italy,Stadio Comunale Chiavari,4154,Césped Artificial
541,Nuova Cosenza,Cosenza,Italy,Stadio San Vito,24209,Natural
542,Pordenone,Pordenone,Italy,NaN,3089,NaN


In [189]:
# Combine the dataframes

result_match_df = pd.merge(result_df, match_info_df, how='inner', on="Link")
combined_df = pd.merge(result_match_df, team_info_df, how='inner', on="Home_Team")

In [193]:
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 136661 entries, 0 to 136660
Data columns (total 22 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Home_Team    136661 non-null  object        
 1   Away_Team    136661 non-null  category      
 2   Result       136661 non-null  string        
 3   Link         136661 non-null  object        
 4   Season       136661 non-null  float64       
 5   Round        136661 non-null  float64       
 6   League       136661 non-null  string        
 7   Home_Goals   136661 non-null  object        
 8   Away_Goals   136661 non-null  object        
 9   Home_Win     136661 non-null  bool          
 10  Away_Win     136661 non-null  bool          
 11  Date_New     136661 non-null  datetime64[ns]
 12  Referee      136661 non-null  object        
 13  Home_Yellow  136661 non-null  int64         
 14  Home_Red     136661 non-null  int64         
 15  Away_Yellow  136661 non-null  int6

In [195]:
path = r'/Users/tom/Documents/Coding/AiCore/Projects/4. Football Match Outcome Predictor /1. EDA and Data Cleaning/combined_df.csv'
combined_df.to_csv(path, index=False)

In [3]:
pd.set_option('display.max_columns', None)
combined_path = r'/Users/tom/Documents/Coding/AiCore/Projects/4. Football Match Outcome Predictor /1. EDA and Data Cleaning/combined_df.csv'
combined_local_df = pd.read_csv(combined_path)

In [6]:
pd.set_option('display.max_columns', None)

combined_local_df

,Home_Team,Away_Team,Result,Link,Season,Round,League,Home_Goals,Away_Goals,Home_Win,Away_Win,Date_New,Referee,Home_Yellow,Home_Red,Away_Yellow,Away_Red,City,Country,Stadium,Capacity,Pitch
0,Watford,Middlesbrough,1-0,https://www.besoccer.com/match/watford-fc/midd...,2021.0,1.0,championship,1,0,True,False,2020-09-11 20:45:00,Stroud K.,1,0,0,0,Watford,England,Vicarage Road Stadium,19920,Natural
1,Watford,Luton Town,1-0,https://www.besoccer.com/match/watford-fc/luto...,2021.0,3.0,championship,1,0,True,False,2020-09-26 13:30:00,Brooks J.,0,0,2,0,Watford,England,Vicarage Road Stadium,19920,Natural
2,Watford,Blackburn Rovers,3-1,https://www.besoccer.com/match/watford-fc/blac...,2021.0,6.0,championship,3,1,True,False,2020-10-21 20:45:00,Woolmer A.,3,0,0,0,Watford,England,Vicarage Road Stadium,19920,Natural
3,Watford,AFC Bournemouth,1-1,https://www.besoccer.com/match/watford-fc/afc-...,2021.0,7.0,championship,1,1,False,False,2020-10-24 13:30:00,Robinson T.,3,0,2,0,Watford,England,Vicarage Road Stadium,19920,Natural
4,Watford,Stoke City,3-2,https://www.besoccer.com/match/watford-fc/stok...,2021.0,10.0,championship,3,2,True,False,2020-11-04 20:00:00,Davies A.,2,0,2,0,Watford,England,Vicarage Road Stadium,19920,Natural
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136656,Castel Sangro,Cesena,1-0,https://www.besoccer.com/match/castel-sangro/a...,1997.0,28.0,serie_b,1,0,True,False,1997-04-06 00:00:00,NaN,0,0,0,0,Castel di Sangro,Italy,NaN,7220,Grass
136657,Castel Sangro,Reggina,1-0,https://www.besoccer.com/match/castel-sangro/r...,1997.0,30.0,serie_b,1,0,True,False,1997-04-20 00:00:00,NaN,0,1,0,0,Castel di Sangro,Italy,NaN,7220,Grass
136658,Castel Sangro,Venezia,1-1,https://www.besoccer.com/match/castel-sangro/v...,1997.0,32.0,serie_b,1,1,False,False,1997-05-04 00:00:00,NaN,0,0,0,0,Castel di Sangro,Italy,NaN,7220,Grass
136659,Castel Sangro,Torino,2-1,https://www.besoccer.com/match/castel-sangro/t...,1997.0,34.0,serie_b,2,1,True,False,1997-05-15 00:00:00,NaN,0,1,0,1,Castel di Sangro,Italy,NaN,7220,Grass
